In [1]:
!python3 -c 'import tensorrt; print("TensorRT version: {}".format(tensorrt.__version__))'

TensorRT version: 8.4.2.4


In [5]:
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt

import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms

import os
import numpy as np
import time

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
IMAGENET_DIR = '/ImageNet'
BATCH_SIZE = 128
LOADER_WORKERS = 4

In [9]:
TRT_LOGGER = trt.Logger()
TRT_MODEL_FILE = '../Flask/Models/resnet34-trt_best.plan'

In [8]:
def load_engine(engine_file_path):
    assert os.path.exists(engine_file_path)
    print("Reading engine from file {}".format(engine_file_path))
    with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

In [10]:
engine = load_engine(TRT_MODEL_FILE)

Reading engine from file ../Flask/Models/resnet34-trt_best.plan


In [25]:
list(engine)

['input_0', 'output_0']

In [29]:
print(engine.get_binding_index('input_0'), engine.get_binding_index('output_0'))
print(engine.get_binding_name(0), engine.get_binding_name(1))
print(engine.get_binding_dtype(0), engine.get_binding_shape(0))
print(engine.get_binding_dtype(1), engine.get_binding_shape(1))

0 1
input_0 output_0
DataType.FLOAT (1, 3, 224, 224)
DataType.FLOAT (1, 1000)


## Inference pipeline

Starting with a deserialized engine, TensorRT inference pipeline consists of the following steps:

- Create an execution context and specify input shape (based on the image dimensions for inference).
- Allocate CUDA device memory for input and output.
- Allocate CUDA page-locked host memory to efficiently copy back the output.
- Transfer the processed image data into input memory using asynchronous host-to-device CUDA copy.
- Kickoff the TensorRT inference pipeline using the asynchronous execute API.
- Transfer the segmentation output back into pagelocked host memory using device-to-host CUDA copy.
- Synchronize the stream used for data transfers and inference execution to ensure all operations are completes.
- Finally, write out the segmentation output to an image file for visualization.

In [33]:
def infer(engine, )

numpy.float32

In [5]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
dataset = torchvision.datasets.ImageNet(root=IMAGENET_DIR, transform=transform, split='val')
loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=LOADER_WORKERS)

In [7]:
n_top1 = 0
n_top5 = 0
cnt = 0

s = time.time()
pred_tms = []
for images, labels in loader:

    ss = time.time()
    output = ort_sess.run([_output], {_input: images.numpy()})
    output = output[0]
    pred_tms.append(time.time()-ss)

    cnt += output.shape[0]

    top1_id = output.argmax(1)
    top5_id = output.argsort(1)[:, ::-1][:, :5]

    n_top1 += np.equal(top1_id, labels).sum()
    n_top5 += np.max(np.isin(top5_id, labels), 1).sum()

    print(f"\rstep: {cnt}/{len(dataset)}", end='')
    
total_tm = time.time() - s
print()
print(f"top-1:  {n_top1/cnt:0.4f}")
print(f"top-5:  {n_top5/cnt:0.4f}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Total Time: {total_tm:0.4f} ({total_tm/len(dataset):0.4f})")
print(f"Average Prediction Time: {np.mean(pred_tms):0.4f}")

step: 50000/50000
top-1:  0.7331
top-5:  0.9218
Batch Size: 128
Total Time: 79.2918 (0.0016)
Average Prediction Time: 0.0528
